In [2]:
import torch
import numpy
import pennylane as qml
import pennylane.numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt

In [3]:
def Cliff2():
    """
    Random 2-qubit Clifford circuit.

    Arguments:
        -nodes (np.ndarray): 
    
    Returns:
        -null
    """
    
    weights = np.random.randint(2, size=(2, 10))
    
    return qml.matrix(qml.RandomLayers(weights=weights,wires=[0,1])).numpy()

In [38]:
def RandomLayers(N_QUBITS, DEPTH):
    """
    Generates brick wall pattern of random 2 qubit Clifford gates

    Arguments:
        -N_QUBITS (int): Number of qubits
        -DEPTH (int): Depth of the circuit

    Returns:
        -random_layers (np.ndarray): Array of 4x4 unitaries (N_QUBITS, DEPTH, 4, 4)
    
    """

    random_layers = []
    for t in range(DEPTH):
        layer = []
        for x in range(0,N_QUBITS,2):
                layer.append(Cliff2())
        random_layers.append(layer)

    return random_layers


In [39]:
N_QUBITS = 2*2
DEPTH = 3

# random_layers = []
# # for t in range(DEPTH):
# #         layer = []
# #         for x in range(0,N_QUBITS,2):
# #                 layer.append(Cliff2())
# #         random_layers.append(layer)

random_layers = RandomLayers(N_QUBITS,DEPTH)


dev = qml.device("default.qubit", wires=N_QUBITS)

@qml.qnode(dev)
def circuit(theta):
    """
    Quantum circuit with random entangling Clifford layers and disentangling layers.
    
    Arguments:
        -theta (np.ndarray): Binary matrix representing the positions of projections. (N_QUBITS, DEPTH)
    
    Returns:
        -Average Von Neumann entropy (float32): Average of 2-qubit Von Neumann entropies over all neighbors.
    """

    theta = theta.T
    DEPTH,N_QUBITS = np.shape(theta)

    for t in range(DEPTH):
        layer = random_layers[t]
        if t%2==0:
            for x in range(0,N_QUBITS,2):
                brick = layer[int(x/2)]
                qml.QubitUnitary(brick,wires=[x,x+1])
        elif t%2==1:
            for x in range(1,N_QUBITS-2,2):
                brick = layer[int((x-1)/2)]
                qml.QubitUnitary(brick,wires=[x,x+1])
            brick = layer[-1]
            qml.QubitUnitary(brick,wires=[N_QUBITS-1,0])
            
        projections = theta[t]
        for x in range(N_QUBITS):
            if projections[x]==1:
                qml.Projector(state=[0],wires=[x])
            
    entropies = []
    for x in range(N_QUBITS-1):
        entropies.append(qml.vn_entropy(wires=[x,x+1]))
    entropies.append(qml.vn_entropy(wires=[N_QUBITS-1,0]))
        
    return entropies

In [40]:
random_layers[0][1]

array([[-0.31538623-0.16805186j,  0.2848996 +0.38242782j,
        -0.31069637+0.67650485j,  0.05876173+0.29534267j],
       [-0.09991234-0.4663007j ,  0.38197083-0.41236206j,
         0.06947403-0.29300784j, -0.18776442+0.57506633j],
       [-0.20140624-0.47645035j, -0.53391572+0.30370361j,
         0.58223989+0.05876173j,  0.00538575+0.11244325j],
       [ 0.61187348-0.05396958j,  0.08444206+0.26870925j,
         0.04189566-0.10448562j,  0.63915461+0.34953096j]])

In [41]:
theta = np.random.randint(2, size=(N_QUBITS,DEPTH))
print(circuit(theta))
drawer = qml.draw(circuit)

print(drawer(theta))

[0.050508910973039285, 0.0505089109730396, 0.05050891097303939, 0.050508910973039604]
0: ─╭U(M0)────────────────╭U(M3)──|0⟩⟨0|─╭U(M4)──|0⟩⟨0|─┤ ╭vnentropy                      
1: ─╰U(M0)──|0⟩⟨0|─╭U(M2)─│──────────────╰U(M4)─────────┤ ╰vnentropy ╭vnentropy           
2: ─╭U(M1)─────────╰U(M2)─│───────|0⟩⟨0|─╭U(M5)──|0⟩⟨0|─┤            ╰vnentropy ╭vnentropy
3: ─╰U(M1)────────────────╰U(M3)─────────╰U(M5)─────────┤                       ╰vnentropy

  ╭vnentropy
  │         
  │         
  ╰vnentropy

M0 = 
[[-0.06705054+0.57216697j  0.05401902+0.00965719j -0.25929892+0.77288436j
  -0.02074693-0.01006343j]
 [-0.03424582-0.00268895j  0.12715046-0.91145472j -0.04664454-0.01368588j
  -0.15532205+0.35413994j]
 [-0.19939869-0.79045957j -0.02074693+0.01006343j  0.02307256+0.57562008j
  -0.05401902+0.00965719j]
 [ 0.04664454-0.01368588j  0.18196495+0.34121663j -0.03424582+0.00268895j
   0.19652089+0.8990531j ]]
M1 = 
[[-0.31538623-0.16805186j  0.2848996 +0.38242782j -0.31069637+0.67650485j
   0.

In [7]:
def RandomFlip(theta,K):
    """
    Randomly flip K entries of a binary matrix theta.

    Arguments:
        -theta (np.ndarray):
        -K (int): 

    Returns:
        -flipped (np.ndarray):

    """
    
    x,y = np.shape(theta)
    N = int(x*y)
    arr = np.array([0] * (N-K) + [1] * K)
    np.random.shuffle(arr)
    arr = arr.reshape(np.shape(theta))

    flipped = (theta + arr) % 2
    
    return flipped

In [31]:
class Disentangler(gym.Env):
    """
    Reinforcement learning environment for the disentangler.

    """
    
    def __init__(self):
        self.action_space = gym.spaces.Discrete(10)
        self.observation_space = []
        self.state = np.zeros((N_QUBITS,DEPTH))
        self.moves = 10

    def step(self, action):
        self.state = RandomFlip(self.state, action)
        self.moves += -1

        entropies = circuit(self.state)
        entropy = sum(entropies)/len(entropies)
        
        if entropy < 1e-16:
            reward = 100
        elif entropy > 1e-16:
            reward = -1

        if self.moves <= 0:
            done = True
        else:
            done = False

        info = {}
        
        return self.state, reward, done, info
    
    def reset(self):
        self.state = np.zeros((N_QUBITS,DEPTH))
        self.moves = 10

        return self.state

In [36]:
env = Disentangler()

In [37]:
episodes = 50
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    
    print('Episode:{} Score:{}'.format(episode, score))


Episode:1 Score:-10
Episode:2 Score:-10
Episode:3 Score:-10
Episode:4 Score:-10
Episode:5 Score:-10
Episode:6 Score:-10
Episode:7 Score:91
Episode:8 Score:-10
Episode:9 Score:-10
Episode:10 Score:91
Episode:11 Score:91
Episode:12 Score:-10
Episode:13 Score:91
Episode:14 Score:-10
Episode:15 Score:91
Episode:16 Score:-10
Episode:17 Score:-10
Episode:18 Score:-10
Episode:19 Score:-10
Episode:20 Score:-10
Episode:21 Score:-10
Episode:22 Score:91
Episode:23 Score:192
Episode:24 Score:-10
Episode:25 Score:192
Episode:26 Score:-10
Episode:27 Score:91
Episode:28 Score:192
Episode:29 Score:-10
Episode:30 Score:192
Episode:31 Score:-10
Episode:32 Score:-10
Episode:33 Score:91
Episode:34 Score:91
Episode:35 Score:-10
Episode:36 Score:-10
Episode:37 Score:91
Episode:38 Score:-10
Episode:39 Score:-10
Episode:40 Score:-10
Episode:41 Score:-10
Episode:42 Score:91
Episode:43 Score:-10
Episode:44 Score:-10
Episode:45 Score:-10
Episode:46 Score:-10
Episode:47 Score:-10
Episode:48 Score:-10
Episode:49 S